In [ ]:
import { ChatMessageHistory} from 'langchain/stores/message/in_memory'
import {HumanMessage,AIMessage} from '@langchain/core/messages'

const history = new ChatMessageHistory()

In [ ]:
await history.addMessage(new HumanMessage('hi'))
await history.addMessage(new AIMessage('what can I do for you?'))

In [ ]:
const messages = await history.getMessages()

console.log(messages);


In [ ]:
import {  Ollama } from "npm:/@langchain/ollama@0.1.5";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";

const chatModel = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "qwen2.5:0.5b",
});

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.
  You are talkative and provides lots of specific details from its context. 
  If the you does not know the answer to a question, it truthfully says you do not know.`,
  ],
  new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);

In [ ]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi, my name is Kai"));

const res1 = await chain.invoke({
  history_message: await history.getMessages(),
});


In [ ]:
res1

In [ ]:
await history.addMessage(new AIMessage(res1))
await history.addMessage(new HumanMessage('what is my name'))

In [ ]:
const res2 = await chain.invoke({
  history_message: await history.getMessages(),
});


In [ ]:
res2

##自动维护chat history

In [ ]:
import { Ollama } from "npm:/@langchain/ollama@0.1.5";
import { ChatOpenAI } from "@langchain/openai";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

// const chatModel = new Ollama({
//   baseUrl: "http://localhost:11434",
//   model: "qwen2.5:0.5b",
// });
const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});


const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. Answer all questions to the best of your ability.",
  ],
  new MessagesPlaceholder("history_message"),
  ["human", "{input}"],
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel);

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});


In [ ]:
const res1 = await chainWithHistory.invoke(
  { input: "hi,my name is Kai" },
  { configurable: { sessionId: "none" } }
);


In [ ]:
res1

In [ ]:
const res2 = await chainWithHistory.invoke(
  { input: "我的名字叫什么" },
  { configurable: { sessionId: "none" } }
);


In [ ]:
res2

In [ ]:
await history.getMessages()

自动生成chat history摘要

In [ ]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";


In [ ]:


const summaryModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const summaryPrompt = ChatPromptTemplate.fromTemplate(`
  Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary
  
  Current summary:
  {summary}
  
  New lines of conversation:
  {new_lines}
  
  New summary:
  `); 

  const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
  ]);



In [ ]:
const newSummary = await summaryChain.invoke({
  summary: "",
  new_lines: "I'm 18",
});


In [ ]:
await summaryChain.invoke({
  summary: newSummary,
  new_lines: "I'm male",
});


In [ ]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";

const summaryModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const summaryPrompt = ChatPromptTemplate.fromTemplate(`
  Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary
  
  Current summary:
  {summary}
  
  New lines of conversation:
  {new_lines}
  
  New summary:
  `);

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
]);

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});
const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.

  Here is the chat history summary:
  {history_summary}
  `,
  ],
  ["human", "{input}"],
]);

let summary = "";
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => history.addUserMessage(input),
    }),
  },
  RunnablePassthrough.assign({
    history_summary: () => summary,
  }),
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      history.addAIMessage(input);
      const messages = await history.getMessages();
      const new_lines = getBufferString(messages);
      const newSummary = await summaryChain.invoke({
        summary,
        new_lines,
      });
      history.clear();
      summary = newSummary;
    },
  }),
]);


In [ ]:
await chatChain.invoke('我饿了')

In [ ]:
await chatChain.invoke('我今天想吃方便面')